In [1]:
# Importing dependencies
import pandas as pd
import numpy as np
import os
import pickle
import random

In [4]:
# Define the path to the CSV file
csv_file = 'hobby_dataset_rated.csv'

# Check if the CSV file exists
if os.path.isfile(csv_file):
    # Load the dataset into a pandas dataframe
    df = pd.read_csv(csv_file)

    # Check if the base_rating column already exists in the dataframe
    if 'base_rating' not in df.columns:
        # Assign a base rating of 15 to each hobby in the dataframe
        df['base_rating'] = 15

        # Save the updated dataset to a CSV file
        df.to_csv('hobby_dataset_v2.csv', index=False)
else:
    # Do nothing if the CSV file doesn't exist
    pass

In [5]:
df.head(2)

,Hobby,Type of Hobby,base_rating
0,Drawing,Creative,15
1,Writing fiction,Creative,15


In [6]:
# Function to get all hobbies of a given type (Private Method)
def get_hobbies_by_type(hobby_type):
    return df[df['Type of Hobby'] == hobby_type]['Hobby'].values

In [ ]:
# Function to randomly suggest a hobby of a given type
def suggest_hobby(hobby_type):
        hobbies = get_hobbies_by_type(hobby_type)
        return np.random.choice(hobbies)
   

In [ ]:
# Function to update the base rating of a hobby based on user feedback
def update_rating(hobby, liked):
    if liked:
        df.loc[df['Hobby'] == hobby, 'base_rating'] += 1
        print(df.loc[df['Hobby'] == hobby]['base_rating'])
    else:
        df.loc[df['Hobby'] == hobby, 'base_rating'] -= 1
        print(df.loc[df['Hobby'] == hobby]['base_rating'])
        if df.loc[df['Hobby'] == hobby, 'base_rating'].values[0] < 0:
            df.drop(df[df['Hobby'] == hobby].index, inplace=True)

    # Save the updated dataset to a CSV file
    df.to_csv('hobby_dataset_v2.csv', index=False)